# Word Sense Disambiguation (снятие лексической неоднозначности) и Word Sense Induction (нахождение значений слова)

**Вам нужно для каждого многозначного слова (т.е. у него есть тэг в первом поле) с помощью алгоритма Леска предсказать нужный синсет и сравнить с правильным. Посчитайте процент правильных предсказаний (accuracy).**

Если считается слишком долго, возьмите поменьше предложений (например, только тысячу)

In [1]:
from lxml import html
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer 


### WSI

## WordNet

In [2]:
# запустите если не установлен ворднет
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Иннокентий\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Домашнее задание. Реализовать алгоритм Леска и проверить его на реальном датасете

Ворднет можно использовать для дизамбигуации. Самый простой алгоритм дизамбигуации - алгоритм Леска. В нём нужное значение слова находится через пересечение слов контекста, в котором употреблено это слово, с определениями значений слова из ворднета. Значение с максимальным пересечением - нужное.

Реализуйте его

In [3]:
def my_lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    if type(text) == str:
        word_list = nltk.word_tokenize(text)
    else:
        word_list = text
    lemmatized_output = [lemmatizer.lemmatize(w) for w in word_list]
    
    return lemmatized_output

In [4]:
def lesk(word, sentence, lemmatize=True):
    
    bestsense = 0
    maxoverlap = 0
    
    synsets = wn.synsets(word)
    if lemmatize:
        lemmatized_sentence = my_lemmatize(sentence)
    else:
        lemmatized_sentence = sentence
    overlaps = {}
    for i, syns in enumerate(synsets):
        definition =  my_lemmatize(syns.definition())
        overlap = len(set(lemmatized_sentence) & set(definition))/len(set(lemmatized_sentence))
        if maxoverlap < overlap:
            bestsense = i
            maxoverlap = overlap          
        
    return bestsense

In [5]:
lesk('day', 'some point or strange period in time') # для примера контекст почти совпадает с одним из определений
# на выходе индекс подходящего синсета

1

In [6]:
# с помощью этого индекса достаем нужный синсет
wn.synsets('day')[1].definition()

'some point or period in time'

**Проверим насколько хорошо работает такой метод на реальном датасете.**

In [7]:
corpus_wsd = []
corpus = open('corpus_wsd_50k.txt').read().split('\n\n')
for sent in corpus:
    corpus_wsd.append([s.split('\t') for s in sent.split('\n')])

Корпус состоит из предложений, где у каждого слова три поля - значение, лемма и само слово. Значение пустое, когда слово однозначное, а у многозначных слов стоит тэг вида **'long%3:00:02::'** Это тэг wordnet'ного формата

In [8]:
corpus_wsd[0]

[['', 'how', 'How'],
 ['long%3:00:02::', 'long', 'long'],
 ['', 'have', 'has'],
 ['', 'it', 'it'],
 ['be%2:42:03::', 'be', 'been'],
 ['', 'since', 'since'],
 ['', 'you', 'you'],
 ['review%2:31:00::', 'review', 'reviewed'],
 ['', 'the', 'the'],
 ['objective%1:09:00::', 'objective', 'objectives'],
 ['', 'of', 'of'],
 ['', 'you', 'your'],
 ['benefit%1:21:00::', 'benefit', 'benefit'],
 ['', 'and', 'and'],
 ['service%1:04:07::', 'service', 'service'],
 ['program%1:09:01::', 'program', 'program'],
 ['', '?', '?']]

In [11]:
%%time
all_ = 0
true = 0

for sent in corpus_wsd[:1000]:
    sentence = []
    for token in sent:
        sentence.append(token[1])
    for token in sent:
        if token[0] != '':
            lemma = token[1]
            true_syns = wn.lemma_from_key(token[0]).synset()
            pred_n = lesk(lemma, sentence, lemmatize=True)
            pred_syns = wn.synsets(lemma)[pred_n]
            all_ += 1
            if true_syns == pred_syns:
                true += 1

Wall time: 41 s


In [12]:
print('accuracy: ', true/all_ )

accuracy:  0.36116236162361626
